In [10]:
!pip install -q imbalanced-learn


In [11]:
import pathlib, glob, os
import pandas as pd, numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB, ComplementNB
import joblib
from pprint import pprint

ROOT = pathlib.Path.cwd()
TRAIN_P = ROOT/"data"/"processed"/"train_dataset.parquet"
MODELS = ROOT/"models"
OUT = ROOT/"data"/"outputs"

MODELS.mkdir(parents=True, exist_ok=True)
OUT.mkdir(parents=True, exist_ok=True)

print("ROOT:", ROOT)
print("TRAIN_P exists:", TRAIN_P.exists())


ROOT: /Users/himanshuagnihotri/sentiment-analysis-project
TRAIN_P exists: True


In [12]:
df = pd.read_parquet(TRAIN_P)
df = df.dropna(subset=["clean","label"]).reset_index(drop=True)
print("rows:", len(df))
print(df['label'].value_counts())
# ensure clean is str
df['clean'] = df['clean'].astype(str)


rows: 608
label
POSITIVE    268
NEGATIVE    215
NEUTRAL     125
Name: count, dtype: int64


In [13]:
X_text = df['clean'].values
y = df['label'].values

try:
    X_train_text, X_test_text, y_train, y_test = train_test_split(
        X_text, y, test_size=0.3, random_state=42, stratify=y)
    print("Used stratified split")
except Exception as e:
    X_train_text, X_test_text, y_train, y_test = train_test_split(
        X_text, y, test_size=0.3, random_state=42)
    print("Stratify failed, used random split:", e)

print("train:", len(X_train_text), "test:", len(X_test_text))


Used stratified split
train: 425 test: 183


In [14]:
vect = TfidfVectorizer(max_features=8000, ngram_range=(1,2), min_df=2)
X_train = vect.fit_transform(X_train_text)
X_test = vect.transform(X_test_text)
print("TF-IDF shapes:", X_train.shape, X_test.shape)
joblib.dump(vect, MODELS/"tfidf_vectorizer_nb.joblib")
print("Saved vectorizer ->", MODELS/"tfidf_vectorizer_nb.joblib")


TF-IDF shapes: (425, 534) (183, 534)
Saved vectorizer -> /Users/himanshuagnihotri/sentiment-analysis-project/models/tfidf_vectorizer_nb.joblib


In [15]:
clf_m = MultinomialNB()
clf_m.fit(X_train, y_train)
y_pred_m = clf_m.predict(X_test)
print("=== MultinomialNB ===")
print("Acc:", accuracy_score(y_test, y_pred_m))
print(classification_report(y_test, y_pred_m, zero_division=0))

clf_c = ComplementNB()
clf_c.fit(X_train, y_train)
y_pred_c = clf_c.predict(X_test)
print("=== ComplementNB ===")
print("Acc:", accuracy_score(y_test, y_pred_c))
print(classification_report(y_test, y_pred_c, zero_division=0))

joblib.dump(clf_m, MODELS/"nb_multinomial.joblib")
joblib.dump(clf_c, MODELS/"nb_complement.joblib")
print("Saved models:", list(MODELS.glob("nb_*.joblib")))


=== MultinomialNB ===
Acc: 0.5683060109289617
              precision    recall  f1-score   support

    NEGATIVE       0.58      0.52      0.55        65
     NEUTRAL       1.00      0.08      0.15        37
    POSITIVE       0.55      0.83      0.66        81

    accuracy                           0.57       183
   macro avg       0.71      0.48      0.45       183
weighted avg       0.65      0.57      0.52       183

=== ComplementNB ===
Acc: 0.5846994535519126
              precision    recall  f1-score   support

    NEGATIVE       0.49      0.75      0.59        65
     NEUTRAL       0.60      0.32      0.42        37
    POSITIVE       0.73      0.57      0.64        81

    accuracy                           0.58       183
   macro avg       0.61      0.55      0.55       183
weighted avg       0.62      0.58      0.58       183

Saved models: [PosixPath('/Users/himanshuagnihotri/sentiment-analysis-project/models/nb_sentiment_vanilla.joblib'), PosixPath('/Users/himanshuagnih

In [16]:
try:
    from imblearn.over_sampling import RandomOverSampler
    ros = RandomOverSampler(random_state=42)
    X_train_dense = X_train.toarray() if hasattr(X_train, "toarray") else X_train
    X_res, y_res = ros.fit_resample(X_train_dense, y_train)
    clf_os = ComplementNB()
    clf_os.fit(X_res, y_res)
    # evaluate on original test set (transform X_test to dense if needed)
    X_test_dense = X_test.toarray() if hasattr(X_test, "toarray") else X_test
    y_pred_os = clf_os.predict(X_test_dense)
    print("=== ComplementNB (oversampled) ===")
    print("Acc:", accuracy_score(y_test, y_pred_os))
    print(classification_report(y_test, y_pred_os, zero_division=0))
    joblib.dump(clf_os, MODELS/"nb_complement_oversample.joblib")
    print("Saved oversample model")
except Exception as e:
    print("Skipping oversample (imblearn missing or error):", e)


=== ComplementNB (oversampled) ===
Acc: 0.5573770491803278
              precision    recall  f1-score   support

    NEGATIVE       0.49      0.72      0.58        65
     NEUTRAL       0.48      0.43      0.46        37
    POSITIVE       0.72      0.48      0.58        81

    accuracy                           0.56       183
   macro avg       0.57      0.55      0.54       183
weighted avg       0.59      0.56      0.56       183

Saved oversample model


In [17]:
vect = joblib.load(MODELS/"tfidf_vectorizer_nb.joblib")
clf = joblib.load(MODELS/"nb_complement.joblib")  # choose model file
examples = [
    "I love this video!",
    "This was awful and fake",
    "The robot performance was impressive",
    "I need help"
]
for t in examples:
    x = vect.transform([t])
    pred = clf.predict(x)[0]
    prob = clf.predict_proba(x).max() if hasattr(clf, "predict_proba") else None
    print(f"{t!r} -> {pred} (conf: {float(prob) if prob is not None else 'n/a'})")


'I love this video!' -> POSITIVE (conf: 0.4806172863118577)
'This was awful and fake' -> NEGATIVE (conf: 0.38365994705903195)
'The robot performance was impressive' -> POSITIVE (conf: 0.37873365706065637)
'I need help' -> NEGATIVE (conf: 0.4021200983729322)


In [18]:
vect = joblib.load(MODELS/"tfidf_vectorizer_nb.joblib")
clf = joblib.load(MODELS/"nb_complement.joblib")

rows = []
# twitter
for f in sorted(glob.glob("data/raw/twitter/*.parquet")):
    dd = pd.read_parquet(f)
    for _, r in dd.iterrows():
        text = str(r.get("clean", r.get("text", "")))
        if not text.strip(): continue
        x = vect.transform([text])
        pred = clf.predict(x)[0]
        conf = float(clf.predict_proba(x).max()) if hasattr(clf, "predict_proba") else None
        created = r.get("created_at")
        # normalize created time to pandas datetime with UTC if present
        try:
            created = pd.to_datetime(created, utc=True)
        except Exception:
            created = created
        rows.append({
            "source":"twitter","file":f,"id":str(r.get("id")),"created_at":created,
            "text": r.get("text"), "clean": text, "pred": pred, "conf": conf
        })

# youtube
for f in sorted(glob.glob("data/raw/youtube/*.parquet")):
    dd = pd.read_parquet(f)
    for _, r in dd.iterrows():
        text = str(r.get("clean", r.get("text", "")))
        if not text.strip(): continue
        x = vect.transform([text])
        pred = clf.predict(x)[0]
        conf = float(clf.predict_proba(x).max()) if hasattr(clf, "predict_proba") else None
        created = r.get("published_at", r.get("created_at"))
        try:
            created = pd.to_datetime(created, utc=True)
        except Exception:
            created = created
        rows.append({
            "source":"youtube","file":f,"id":str(r.get("id")),"created_at":created,
            "text": r.get("text"), "clean": text, "pred": pred, "conf": conf
        })

out = pd.DataFrame(rows)
out_fn = OUT / f"predictions_nb_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.parquet"
out.to_parquet(out_fn, index=False)
print("Saved:", out_fn)
print(out.groupby(["source","pred"]).size().unstack(fill_value=0))


Saved: /Users/himanshuagnihotri/sentiment-analysis-project/data/outputs/predictions_nb_20251128_002246.parquet
pred     NEGATIVE  NEUTRAL  POSITIVE
source                              
twitter         5        0         0
youtube       251       48       201


In [19]:
files = sorted(glob.glob("data/outputs/predictions_nb_*.parquet"))
if not files:
    print("No predictions found")
else:
    p = files[-1]
    print("Using:", p)
    d = pd.read_parquet(p)
    display(d.groupby(["source","pred"]).size().unstack(fill_value=0))
    display(d.sort_values("conf", ascending=False).head(40))


Using: data/outputs/predictions_nb_20251128_002246.parquet


pred     NEGATIVE  NEUTRAL  POSITIVE
source                              
twitter         5        0         0
youtube       251       48       201

source                                               file  \
268  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
262  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
270  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
260  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
261  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
274  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
263  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
265  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
266  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
267  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
269  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
272  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
36   youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
30   youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
31   youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
32   youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
34   youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
35   youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
29   youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
27   youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
329  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
413  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
291  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
243  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
480  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
460  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
471  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
416  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
78   youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
92   youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
74   youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
113  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
16   youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
169  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
309  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
294  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
106  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
323  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
341  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   
241  youtube  data/raw/youtube/youtube_comments_Ah_uuTwGOYU_...   

                             id                created_at  \
268  UgwqGaEUsxAGRtTuf-V4AaABAg 2025-11-26 16:27:32+00:00   
262  Ugww_9tNeMV-Rwtuv_B4AaABAg 2025-11-26 16:29:17+00:00   
270  Ugy7AQBNKqUMIzLZTfp4AaABAg 2025-11-26 16:26:39+00:00   
260  UgyITXgfjrIfqydNk7Z4AaABAg 2025-11-26 16:29:56+00:00   
261  Ugx6cRP9ZkPaczmHI494AaABAg 2025-11-26 16:29:34+00:00   
274  UgyOYmvfi0l_WickM094AaABAg 2025-11-26 16:25:24+00:00   
263  UgwSAnHHt66nGPok_UB4AaABAg 2025-11-26 16:28:58+00:00   
265  UgxvssQVLh-YkLfKN7J4AaABAg 2025-11-26 16:28:37+00:00   
266  UgzSWxfja9cvfFHImIV4AaABAg 2025-11-26 16:28:16+00:00   
267  UgyJ6shUuh_yKD2zyUd4AaABAg 2025-11-26 16:27:57+00:00   
269  Ugya7YFOrwlVS3sQY2x4AaABAg 2025-11-26 16:27:09+00:00   
272  UgycffbMzWpsbTwGg1d4AaABAg 2025-11-26 16:26:12+00:00   
36   UgxW53LKqZxQOHbC7fx4AaABAg 2025-11-26 19:10:56+00:00   
30   UgwMz7kMAJE4P3ZqK_F4AaABAg 2025-11-26 19:12:50+00:00   
31   UgyWA3pHa6t7vhMNbPp4AaABAg 2025-11-26 19:12:21+00:00   
32   UgwNaqyPeYkpY3DwAed4AaABAg 2025-11-26 19:12:11+00:00   
34   UgxIjPH7WTzqVpm2wNF4AaABAg 2025-11-26 19:11:22+00:00   
35   UgwCUzyOEck2OQ-EEgZ4AaABAg 2025-11-26 19:11:11+00:00   
29   UgxsmSqXtCe1Oqbphwl4AaABAg 2025-11-26 19:13:07+00:00   
27   Ugx4TiReLAYB0UprktV4AaABAg 2025-1

In [2]:
import seaborn as sns

def plot_conf_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred, labels=["NEGATIVE","NEUTRAL","POSITIVE"])
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, cmap="Blues", fmt="d",
                xticklabels=["NEG","NEU","POS"],
                yticklabels=["NEG","NEU","POS"])
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()


In [3]:
def plot_roc(model, X, y_true, title):
    classes = ["NEGATIVE","NEUTRAL","POSITIVE"]
    y_bin = label_binarize(y_true, classes=classes)

    y_score = model.predict_proba(X)
    plt.figure(figsize=(6,5))

    for i, cls in enumerate(classes):
        fpr, tpr, _ = roc_curve(y_bin[:, i], y_score[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{cls} (AUC={roc_auc:.2f})")

    plt.plot([0, 1], [0, 1], "k--")
    plt.title(f"ROC Curve — {title}")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend()
    plt.show()


In [5]:
models = {
    "Vanilla Logistic": (m_vanilla, X_main),
    "Weighted Logistic": (m_weighted, X_main),
    "Oversampled Logistic": (m_over, X_main),
    "Naive Bayes": (m_nb, X_nb)
}

for name, (model, X) in models.items():
    print("\n===============================")
    print(f"📌 MODEL: {name}")
    print("===============================")

    y_pred = model.predict(X)
    
    print("Accuracy:", accuracy_score(y, y_pred))
    print(classification_report(y, y_pred))
    
    plot_conf_matrix(y, y_pred, f"Confusion Matrix — {name}")
    plot_roc(model, X, y, f"ROC — {name}")


NameError: name 'm_nb' is not defined